In [1]:
import pandas as pd

In [2]:
# Estimacion del precio mensual de locales commerciales por barrio en años 2011, 2010, 2009, 2008
df1 = pd.read_csv('../Datos CSV/locveevolucio.csv')
relabel = {'BARIIS': 'BARRIO', 'DOSZEROONZE': 2011, 'DOSZERODEU': 2010, 'DOSZEROZERONOU': 2009, 'DOSZEROZEROVUIT': 2008}
df1.rename(columns=relabel, inplace=True)

In [3]:
df1['BARRIS'] = df1['BARRIS'].str.replace('[\.,0-9]', '')
# df1.to_json('data/evolucion_precio_renta_mensual.json', orient='records')

/var/folders/hp/ytf1kqk92hd11vlmtk9zz38r0000gn/T/ipykernel_82537/3902438200.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['BARRIS'] = df1['BARRIS'].str.replace('[\.,0-9]', '')


In [5]:
# Remove punctuations and also add column for average price, high price, low price
df1.head(5)

,DTE,BARRIS,2011,2010,2009,2008
0,1,el Raval,2.176,2.445,2.773,3.065
1,1,el Barri Gòtic,2.632,3.026,3.750,3.828
2,1,la Barceloneta,2.135,3.005,--,--
3,1,Sant Pere Santa Caterina i la Ribera,2.482,2.828,3.146,3.450
4,2,el Fort Pienc,2.224,2.422,2.736,2.893


### Censo Comercial Dataset

In [251]:
df2 = pd.read_csv('../Datos CSV/2019_censcomercialbcn_detall.csv')

/Users/joaquinortega/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (35,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [252]:
# direccion = 'Codi_Via', 'Nom_Via', 'Planta', 'Porta','Lletra_Inicial'
censo_cols = ['ID_Bcn_2019','Nom_Principal_Activitat','Nom_Activitat','Nom_Sector_Activitat','Nom_Grup_Activitat','Nom_Local','Latitud','Longitud','Nom_Districte']
censo_clean = pd.DataFrame(df2[censo_cols])
censo_clean.rename(columns={'Nom_Principal_Activitat':'Actividad Economica','Nom_Activitat':'Nombre de Actividad'}, inplace=True)
censo_clean['Actividad Economica'].replace({'Actiu': 'For Profit', 'Sense activitat Econòmica': 'Non Profit'}, inplace=True)

In [255]:
censo_clean['Nombre de Actividad'].value_counts()

Sense informació                                     11164
Restaurants                                           5599
Sense informació - Havia tingut activitat (rètol)     4276
Serveis a les empreses i oficines                     3464
Bars   / CIBERCAFÈ                                    3290
                                                     ...  
Souvenirs i basars                                      40
Òptiques i fotografia                                   27
Grans magatzems i hipermercats                          20
altres                                                  16
Altres ( per exemple VENDING)                           16
Name: Nombre de Actividad, Length: 83, dtype: int64

In [256]:
first_lambda = ['Sense informació','altres','Altres ( per exemple VENDING)', 'Altres']

In [257]:
censo_clean['Nombre de Actividad'].value_counts()

Sense informació                                     11164
Restaurants                                           5599
Sense informació - Havia tingut activitat (rètol)     4276
Serveis a les empreses i oficines                     3464
Bars   / CIBERCAFÈ                                    3290
                                                     ...  
Souvenirs i basars                                      40
Òptiques i fotografia                                   27
Grans magatzems i hipermercats                          20
altres                                                  16
Altres ( per exemple VENDING)                           16
Name: Nombre de Actividad, Length: 83, dtype: int64

In [258]:
# Cambiar con la tercera columna - Nom_Grup_Activitat = First Lambda
censo_clean['Nombre de Actividad'] = censo_clean.apply(lambda row: row['Nom_Grup_Activitat'] if row['Nombre de Actividad'] in first_lambda else row['Nombre de Actividad'], axis=1)

In [259]:
censo_clean['Nombre de Actividad'].value_counts()

Grup no definit                                                   11164
Restaurants                                                        5599
Sense informació - Havia tingut activitat (rètol)                  4276
Serveis a les empreses i oficines                                  3464
Bars   / CIBERCAFÈ                                                 3290
                                                                  ...  
Maquinària                                                           67
Souvenirs i basars                                                   40
Restaurants, bars i hotels (Inclòs hostals, pensions i fondes)       32
Òptiques i fotografia                                                27
Grans magatzems i hipermercats                                       20
Name: Nombre de Actividad, Length: 83, dtype: int64

In [260]:
# Cambiar con la segunda columna - Nom_Sector_Activitat
censo_clean['Nombre de Actividad'] = censo_clean.apply(lambda row: row['Nom_Sector_Activitat'] if row['Nombre de Actividad'] in first_lambda else row['Nombre de Actividad'], axis=1)

In [265]:
pd.set_option('display.max_rows', None)
censo_clean['Nombre de Actividad'].value_counts()

Grup no definit                                                   11164
Restaurants                                                        5599
Sense informació - Havia tingut activitat (rètol)                  4276
Serveis a les empreses i oficines                                  3464
Bars   / CIBERCAFÈ                                                 3290
Vestir                                                             3157
Perruqueries                                                       2659
Autoservei / Supermercat                                           2337
Ensenyament                                                        2115
Sanitat i assistència                                              2024
Activitats de la construcció                                       1965
Pa, pastisseria i làctics                                          1562
Centres d'estètica                                                 1499
Activitats emmagatzematge                                       

In [268]:
censo_clean.to_json('../JSON listos/censo_clean.json', orient='records')  

### CSV Centro Comercial

In [130]:
# Load the CSV - Centro Comercial
centros_comerciales = pd.read_csv('../Datos CSV/opendatagrans-centres-comercials.csv', encoding='utf-16')

In [131]:
# Add new column for categorization
centros_comerciales['Tipo_Centro_Comercio'] = 'Centro Comercial'

### Galerias Comerciales

In [132]:
# Load the CSV - Galerias Comerciales
galerias = pd.read_csv('../Datos CSV/opendata_galeries-comercials.csv', encoding='utf-16')

In [133]:
# Add new column for categorization - Galerias Comerciales
galerias['Tipo_Centro_Comercio'] = 'Galeria Comercial'

### Mercats Fires al Carrer

In [134]:
# Add new column for categorization - Galerias Comerciales
mercats_fires = pd.read_csv('../Datos CSV/opendata_mercats-fires-al-carrer.csv', encoding='utf-16')

In [135]:
mercats_fires['Tipo_Centro_Comercio'] = 'Feria'

### Mercados Municipales

In [136]:
mercats_municipals = pd.read_csv('../Datos CSV/opendata_mercats-municipals.csv', encoding='utf-16')

In [137]:
mercats_municipals['Tipo_Centro_Comercio'] = 'Mercado Municipal'

### Concatenation of Centros de Comercio

In [138]:
comercio_bcn = pd.concat([centros_comerciales, galerias, mercats_fires, mercats_municipals])

In [146]:
comercio_cols = ['register_id','name','addresses_road_name','addresses_start_street_number','addresses_district_name','addresses_zip_code','geo_epgs_4326_x','geo_epgs_4326_y','Tipo_Centro_Comercio']
filtered_comercio = pd.DataFrame(comercio_bcn[comercio_cols])
filtered_comercio

,register_id,name,addresses_road_name,addresses_start_street_number,addresses_district_name,addresses_zip_code,geo_epgs_4326_x,geo_epgs_4326_y,Tipo_Centro_Comercio
0,﻿1256130510,Centre Comercial Som Multiespai,Av Rio de Janeiro,42,Nou Barris,8016,41.435125,2.181042,Centro Comercial
1,﻿1313140637,Centre Comercial Diagonal Mar,Av Diagonal,3,Sant Martí,8019,41.410457,2.216423,Centro Comercial
2,﻿95111123350,Hipermercat Carrefour *Glòries,Av Diagonal,208,Sant Martí,8018,41.404976,2.192521,Centro Comercial
3,﻿93308093528,Centre Comercial L'Illa Diagonal,Av Diagonal,557,Les Corts,8029,41.389746,2.135396,Centro Comercial
4,﻿93293093019,Centre Comercial Westfield Glòries,Av Diagonal,208,Sant Martí,8018,41.404976,2.192521,Centro Comercial
...,...,...,...,...,...,...,...,...,...
46,﻿92086019717,Mercat de Canyelles,Carrer d'Antonio Machado,10,Nou Barris,8042,41.441988,2.164427,Mercado Municipal
47,﻿96058202955,Mercat Dominical de Sant Antoni,C Comte d'Urgell,1,Eixample,8011,41.379568,2.161743,Mercado Municipal
48,﻿96058202955,Mercat Dominical de Sant Antoni,C Comte d'Urgell,1,Eixample,8011,41.379568,2.161743,Mercado Municipal
49,﻿99400695137,Mercat de Núria,Avinguda dels Rasos de Peguera,186,Nou Barris,8033,41.459952,2.172637,Mercado Municipal
